In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import numpy as np

In [ ]:
data_path = 'data/'

In [ ]:
cities = pd.read_parquet(data_path + 'cities.parquet')
sales = pd.read_parquet(data_path + 'sales.parquet')
shops = pd.read_parquet(data_path + 'shops.parquet')

# Анализируем продажы

In [ ]:
sales.head()

In [ ]:
len(sales['shop_id'].unique())

Есть информация о 845 магазинах

In [ ]:
sales['goods_type'].unique()

Нормальный такой наборчик

In [ ]:
sales.groupby('shop_id').apply(lambda x: len(x['number_of_counters'].unique()) == 1).value_counts()

В некоторых магазинах количество прилавков одинаковое, в некоторых - нет

In [ ]:
sales.groupby(['shop_id', 'date']).apply(lambda x: len(x['number_of_counters'].unique()) == 1).value_counts()

Количество прилавков одинаковое в определенном магазине в опредленный день для всех товаров

Приведем данные к широкому формату

In [ ]:
sales_pivoted = pd.pivot_table(sales[['date', 'shop_id', 'goods_type', 'total_items_sold']], index = ['date', 'shop_id'], columns = 'goods_type')

In [ ]:
sales_pivoted.columns = sales_pivoted.columns.get_level_values(1)

In [ ]:
sales_pivoted = sales_pivoted.reset_index()

In [ ]:
sales_pivoted.head()

Добавим информацию о количестве работающих прилавков

In [ ]:
sales = sales[['date', 'shop_id', 'number_of_counters']].drop_duplicates(['date', 'shop_id']).reset_index(drop = True)

In [ ]:
sales_pivoted = pd.merge(sales_pivoted, sales, on = ['date', 'shop_id'])

In [ ]:
sales_pivoted.head()

In [ ]:
sales_pivoted['date'].max()

In [ ]:
sales_pivoted.groupby('shop_id')['date'].max().sort_values()

In [ ]:
sales_pivoted['date'].min()

In [ ]:
sales_pivoted.groupby('shop_id')['date'].min().sort_values()

Для магазинов есть наблюдения с максимума 2146-01-01 до минимум 2147-11-30. Ограничим выборку данным промежутком чтобы все ряды были одинаковой длины. Это упростит дальнейший анализ

Проверим, есть ли пропущенные дни в данных о магазинах

In [ ]:
def find_gaps(x):
    x['date_shifted'] = x['date'].shift(1)
    x['date_previous'] = x['date'] - pd.Timedelta('1 day')
    return x

In [ ]:
sales = sales.sort_values(['shop_id', 'date'])
sales = sales.groupby('shop_id').apply(find_gaps).reset_index(drop = True)

In [ ]:
sales[sales['date_shifted'] != sales['date_previous']]

In [ ]:
sales.iloc[615536-5:615536+5, :]

Например в магазине 2 пропущены 28 и 29 числа. Заполним пропущенные даты.

In [ ]:
shops

In [ ]:
def filling_gaps(x):
    dates = list(pd.date_range('2146-01-01 00:00:00', '2147-11-30 00:00:00'))
    dates = pd.DataFrame({'date': dates})
    x = pd.merge(x, dates, on = 'date', how = 'right')
    return x

In [ ]:
sales_pivoted = sales_pivoted.groupby('shop_id').apply(filling_gaps).reset_index(drop = True)

In [ ]:
sales_pivoted.head()

Будем считать, что если пропущена информация о продажах, то товары в этот день не продавались

In [ ]:
sales_pivoted = sales_pivoted.fillna(0)
sales_pivoted.head()

In [ ]:
palette = sns.color_palette("tab10")

In [ ]:
sampled_shops = np.random.choice(shops['shop_id'].values, 10, replace = False)

In [ ]:
sampled_shops

In [ ]:
fig, ax = plt.subplots(6, 2, figsize = (30, 60))
row = 0
col = 0
for col_name in sales_pivoted.columns[2:]:
    _ = sns.lineplot(x = 'date', 
                     y = col_name, 
                     data = sales_pivoted[sales_pivoted['shop_id'].isin(sampled_shops)],
                     hue = 'shop_id',
                     palette = palette, 
                     ax = ax[row][col])
    if col == 1:
        row = row+1
        col = 0
    else:
        col = col+1

In [ ]:
sales_pivoted.head()

Приведем данные от абсолютных продаж к данным о продажах на 1 прилавок

In [ ]:
goods = sales_pivoted.iloc[:, 2:-1].columns

In [ ]:
for good in goods:
    sales_pivoted[good] = sales_pivoted[good]/sales_pivoted['number_of_counters']

In [ ]:
sales_pivoted[sales_pivoted['number_of_counters'] == 0].head()

Заменим пропущенные значения нулями

In [ ]:
sales_pivoted = sales_pivoted.fillna(0)

Заменим абсолютные значения на процентное измнение по сравнению с предыдущим днем

In [ ]:
sales_pivoted = sales_pivoted.sort_values(['shop_id', 'date'])

In [ ]:
for good in goods:
    print(good)
    sales_pivoted[good] = sales_pivoted.groupby('shop_id')[good]\
                            .rolling(2)\
                            .apply(lambda x: (x.iloc[1] - x.iloc[0])/x.iloc[0]).values

In [ ]:
sales_pivoted.tail()

In [ ]:
sales_pivoted = sales_pivoted.replace(np.inf, np.nan)

In [ ]:
sales_pivoted.head()

In [ ]:
benz_sales = pd.pivot_table(sales_pivoted[['shop_id', 'Бензак', 'date']], index = 'shop_id', values = 'Бензак', columns = 'date')

In [ ]:
benz_sales = benz_sales.reset_index(drop = True)

In [ ]:
benz_sales.columns.name = ''

In [ ]:
clust_model = KMeans(n_clusters = 10)

In [ ]:
benz_sales

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
clust_model = KMeans(n_clusters = 10,n_init=500,  max_iter = 2000)
clust_model.fit(benz_sales.dropna())
pd.Series(clust_model.labels_).value_counts()

In [ ]:
_ = plt.figure(figsize = (10, 10))
sns.lineplot(x = range(0, benz_sales.shape[1]), y = benz_sales.iloc[0, :].values)
sns.lineplot(x = range(0, benz_sales.shape[1]), y = benz_sales.iloc[1, :].values)
sns.lineplot(x = range(0, benz_sales.shape[1]), y = benz_sales.iloc[2, :].values)
sns.lineplot(x = range(0, benz_sales.shape[1]), y = benz_sales.iloc[3, :].values)

In [ ]:
sales_pivoted[sales_pivoted['shop_id'] == 0]['Бензак'].values
_ = plt.figure(figsize = (10, 10))
sns.lineplot(x = range(0, benz_sales.shape[1]), y = benz_sales.iloc[0, :].values)

По графикам видно, что ряды продаж у разных магазинов выглядят по разному. При этом это ряды не являются вертикальными сдвигами друг друга, то есть отличается не только объем продаж, но и его распределение по времени.

Есть небольшой возрастающий тренд

Будем разбивать магазины на группы, в которых временные ряды продаж ведут себя примерно одинаково. Будем рассматривать относительные величины продаж, а не абсолютные, т.к. в случае рассмотрения абсолютных величин мы получим кластеры, которым пренадлежат магазины с похожими объемами продаж товаров. Это не несет бизнес ценности, т.к. несмотря на похожие объемы, структура распределения продаж во времени может быть совершенно разной